# IMPORTANT!

1. You need to have created a .env file -> add your url, token, and course_id
2. This process allows you to select an assignment and a group set from your course
3. You can then N peer reviews that will be applied within the groups in the selected group set
> Note: you must set N to at least 1 less than members in the groups
4. ⚠️ Running this will **delete ALL current peer reviews** for the assignment (this cannot be undone)
5. The peer review assignment is random, so when you run will be different each time

▶️ When you are ready select Kernel -> Restart & Run All.

Follow the steps below (select assignment, select groups, and n peer reviews). 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import my_app
import button_tests

my_app.my_app()



Token Valid: Alison Myers (185698)



Accessed submissions for Assignment with no due date (PR)
Deleted all peer reviews for Assignment with no due date (PR)
Accessing submissions for group: TEST2 1
4: dict_values([[93991, 93994], [93994, 93991]])
93991 will be reviewed by: [93994]
93994 will be reviewed by: [93991]
Accessing submissions for group: TEST2 2
2: dict_values([[93995, 93992], [93992, 93995]])
93995 will be reviewed by: [93992]
93992 will be reviewed by: [93995]
Accessing submissions for group: TEST2 3
4: dict_values([[93997, 93999], [93999, 93997]])
93997 will be reviewed by: [93999]
93999 will be reviewed by: [93997]
